In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

Function below scrape the play by play data using the numbers to the link of the screenshot below.

In [3]:
def play2play(matchnumber):
    link = "https://stats.ncaa.org/game/play_by_play/" + matchnumber
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    page = requests.get(link, headers = headers).text.strip()
    results = re.findall("smtext\">.+?</td>",page)
    trimlist = []
    for i in range(len(results)):
        trim = results[i].strip("smtext\">" + "</td")
        trimlist.append(trim)
    return trimlist

In [5]:
#play2play("5212037")

# STARTS HERE

In [4]:
#Starts with extracting the team name and their number and putting them into a dataframe
link = "https://stats.ncaa.org/selection_rankings/nitty_gritties/27163"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
page = requests.get(link, headers = headers).text.strip()
results = re.findall("<a target=\"TEAM_PAGE\" class=\"skipMask\" href=\"/teams/.*?</a>",page)
teamnumber = []
for i in range(len(results)):
        trim = results[i].strip("<a target=\"TEAM_PAGE\" class=\"skipMask\" href=\"/teams/")
        teamnumber.append(trim)
number = []
for i in range(len(teamnumber)):
    number.append(teamnumber[i][0:6])

teamname = []
for i in range(len(teamnumber)):
    teamname.append(teamnumber[i][8:(len(teamnumber[i])-4)])

df = pd.DataFrame({'Team' : teamname, 'TeamNumber' : number})    

In [5]:
#Output of the table
df

,Team,TeamNumber
0,Villanova,525678
1,Kennesaw St.,525688
2,High Point,525690
3,Pittsburgh,535118
4,American,525581
...,...,...
113,Loyola Maryland,525634
114,Marist,525636
115,New Hampshire,525646
116,Rutgers,525660


In [7]:
#Finding the game by game link to eventually lead to the box-to-box play
game_by_game_link = []
for i in range(len(number)):
    link = "https://stats.ncaa.org/teams/" + number[i]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    page = requests.get(link, headers = headers).text.strip()
    results = (re.findall("<a href=\"(.*?)\">Game By Game</a>",page))
    results = "https://stats.ncaa.org/" + results[0]
    game_by_game_link.append(results)

In [9]:
#This game_by_game link return the 2021-2022 page for each teams according to the order in the dataframe above.
game_by_game_link

In [10]:
boxlinkall = []
for i in range(len(game_by_game_link)):
    link = game_by_game_link[i]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    page = requests.get(link, headers = headers).text.strip()
    results = re.findall("skipMask\" href=\".+?box_score\"",page)
    trimlist = []
    for i in range(len(results)):
        trim = results[i].strip("skipMask\" href=\"") + 're'
        trimlist.append(trim)
    boxlink = []
    for i in range(len(trimlist)):
        nlink = "https://stats.ncaa.org" + trimlist[i]
        boxlink.append(nlink)
    boxlinkall.append(boxlink)

In [80]:
#the link are in list of a list. Ex, [[1,2],[1,2],...] with each of them [1,2] represent the season box score for a team
#boxlinkall

In [12]:
#Unlisted the link into a single list to help data preparation
boxlinkall_unlist = []
for item in boxlinkall:
    # appending elements to the boxlinkall_unlist
    boxlinkall_unlist += item

In [13]:
boxlinkall_unlist

['https://stats.ncaa.org/contests/2211838/box_score',
 'https://stats.ncaa.org/contests/2211908/box_score',
 'https://stats.ncaa.org/contests/2211969/box_score',
 'https://stats.ncaa.org/contests/2212090/box_score',
 'https://stats.ncaa.org/contests/2212394/box_score',
 'https://stats.ncaa.org/contests/2212594/box_score',
 'https://stats.ncaa.org/contests/2212849/box_score',
 'https://stats.ncaa.org/contests/2213045/box_score',
 'https://stats.ncaa.org/contests/2213240/box_score',
 'https://stats.ncaa.org/contests/2213535/box_score',
 'https://stats.ncaa.org/contests/2213814/box_score',
 'https://stats.ncaa.org/contests/2214224/box_score',
 'https://stats.ncaa.org/contests/2214710/box_score',
 'https://stats.ncaa.org/contests/2215142/box_score',
 'https://stats.ncaa.org/contests/2215481/box_score',
 'https://stats.ncaa.org/contests/2215901/box_score',
 'https://stats.ncaa.org/contests/2211837/box_score',
 'https://stats.ncaa.org/contests/2212052/box_score',
 'https://stats.ncaa.org/con

In [81]:
matches = []
for i in range(len(boxlinkall_unlist)):
    link = boxlinkall_unlist[i]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    page = requests.get(link, headers = headers).text.strip()
    gamedata = re.findall("<td colspan=\"26\">(.*?)</td>",page)
    start_1 = re.search("<td width=\"20%\">\nTotals", page)
    end_1 = re.search("</tr>\n\n\n\n</table>\n\n\n<br/>\n\n<table class=\"mytable\"", page)
    firststatstart = start_1.end()
    firststatend = end_1.start()

    second = page[firststatend:]
    start_2 = re.search("<td width=\"20%\">\nTotals", second)
    end_2 = re.search("<div class=\"footer\"></div>", second)
    secondstatstart = start_2.end()
    secondstatend = end_2.start()

    data_1 = page[firststatstart:firststatend]
    data_2 = second[secondstatstart:secondstatend]
    stat_final1 = re.findall("<td style=\"text-align:right\"  data-order=\"(.*?)\">",data_1)
    stat_final2 = re.findall("<td style=\"text-align:right\"  data-order=\"(.*?)\">",data_2)

    gamedata.extend(stat_final1)
    gamedata.extend(stat_final2)
    
    matches.append(gamedata)

In [83]:
df_draft = pd.DataFrame (matches, columns = ['Team 1', 'Team 2', 'Fouls','GS','Goals','Assists','Points','Shots','SOG','Ground Balls','Turnovers','CT',
                                                   'FOs Won','FOs Taken','G Min','Goals Allowed','Saves','W','L','Freepos Shots','Freepos Goals','RC','YC','Draw Controls'
                                                   ,'Clears','Clr Att','Fouls_T2','GS_T2','Goals_T2','Assists_T2','Points_T2','Shots_T2','SOG_T2','Ground Balls_T2','Turnovers_T2','CT_T2',
                                                   'FOs Won_T2','FOs Taken_T2','G Min_T2','Goals Allowed_T2','Saves_T2','W_T2','L_T2','Freepos Shots_T2','Freepos Goals_T2','RC_T2','YC_T2','Draw Controls_T2'
                                                   ,'Clears_T2','Clr Att_T2'])

In [85]:
df_draft.to_csv('df_draft.tsv', sep="\t") 

# EXTRA CODE NOT YET DONE(play-by-play text)

In [134]:
link = boxlinkall[0][0]
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
page = requests.get(link, headers = headers).text.strip()
results = re.findall("<li>\n\t\t\t<a href=\"/game/play_by_play/(.*?)\">Play by Play</a>",page)
p2p_link = "https://stats.ncaa.org/game/play_by_play/" + results[0]
p2p_link

'https://stats.ncaa.org/game/play_by_play/5227085'

In [136]:
p2p_all = []
for i in range(len(boxlinkall[0])):
    link = boxlinkall[0][i]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    page = requests.get(link, headers = headers).text.strip()
    results = re.findall("<li>\n\t\t\t<a href=\"/game/play_by_play/(.*?)\">Play by Play</a>",page)
    p2p_link = "https://stats.ncaa.org/game/play_by_play/" + results[0]
    p2p_all.append(p2p_link)
    

In [138]:
p2p_all

['https://stats.ncaa.org/game/play_by_play/5227085',
 'https://stats.ncaa.org/game/play_by_play/5206259',
 'https://stats.ncaa.org/game/play_by_play/5208699',
 'https://stats.ncaa.org/game/play_by_play/5213187',
 'https://stats.ncaa.org/game/play_by_play/5221040',
 'https://stats.ncaa.org/game/play_by_play/5230078',
 'https://stats.ncaa.org/game/play_by_play/5230693',
 'https://stats.ncaa.org/game/play_by_play/5233270',
 'https://stats.ncaa.org/game/play_by_play/5237260',
 'https://stats.ncaa.org/game/play_by_play/5240576',
 'https://stats.ncaa.org/game/play_by_play/5246029',
 'https://stats.ncaa.org/game/play_by_play/5257453',
 'https://stats.ncaa.org/game/play_by_play/5262579',
 'https://stats.ncaa.org/game/play_by_play/5266636',
 'https://stats.ncaa.org/game/play_by_play/5272123',
 'https://stats.ncaa.org/game/play_by_play/5281512']